<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Use Keras to predict credit card fraud</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="http://www.thebluediamondgallery.com/typewriter/images/credit-card-fraud.jpg" width="600" alt="Icon"/>        </th>
   </tr>
</table>

This notebook contains the steps and code to create a model that detects credit card fraud. You will use an anonymized data set of credit card transactions,  which is available at Kaggle. The sample data set consists of 285K credit card transactions each with 31 columns. Most of the features of the  data set are the result of PCA except for the Time and Amount. The Time column represente the elapsed time in seconds relative to the  first row in the dataset and the  Amount column represents the dollar amount of each transaction. 

The data is highly unbalanced as 98.828% are normal transactions and only 0.172% are fraudulent. This means that the overall accuracy of any model you build will not be a good measure of the model's performance. If you guess that every transaction is normal 100% of the time you would be right 99.828% of the time ! 

You will buld a Logistic Regression model in Keras, which may seem unlikely at first,  as Keras is a typically used for Deep Learning networks,  however Logistic Regression is just a single layer neural network with no hidden layers, so Keras makes this very easy to build. 

## Learning goals

In this notebook, you will learn how to:

-  Load a sample data set from ``Kaggle``.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create a Keras model.
-  Train and evaluate the  model.


<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

- Download the credit card data from [here](https://www.kaggle.com/mlg-ulb/creditcardfraud). The name of the file is *creditcard.csv*
- Click on the data icon at the top right of the notebook window and then select and upload the *creditcard.csv* file.
![Data icon](https://github.com/djccarew/ccfraud-keras-lab-part1/raw/master/images/ss5.png)
- Once the file is uploaded, place your cursor in the code cell below and select **Insert to code->Insert pandas Dataframe**
![Insert code](https://github.com/djccarew/ccfraud-keras-lab-part1/raw/master/images/ss6.png) 

In [ ]:
# With your cursor in this cell, insert the code to read the dataset into a 
# DataFrame as instructed above


<a id="load"></a>
## 2. Load and explore data

In this section you will load the data from scikit-learn sample data sets and then perform a basic exploration.

In [ ]:
# New copy of the imported Dataframe
# Make sure variable name on the right of this assigment statement matches the value inserted
# into the code cell above
data =  df_data_1.copy()
# The  Time column does  not correlate with either fradulent or normal transactions so we'll drop it 
data = data.drop(['Time'], axis=1)

In the next step, count the data examples.

In [ ]:
# Calculate the number of samples.
print("Number of normal transactions: ", data[data.Class == 0].shape[0])
print("Number of fraudulent transactions: ", data[data.Class == 1].shape[0])
print("Total transactions: " + str(len(data.index)))

<a id="model"></a>
## 3. Create a Keras model

In this section you learn how to:
- [Prepare the data](#prep)
- [Create the Keras machine learning model](#pipe)
- [Train the model](#train)

### 3.1 Prepare the data<a id="prep"></a>

Standardize the features by removing the mean and scaling to unit variance.

In [ ]:
from sklearn.preprocessing import StandardScaler
data_standardized = data.copy()
for col_name in data_standardized.columns.values:
    if col_name != 'Class':
        data_standardized[col_name] = StandardScaler().fit_transform(data_standardized[col_name].values.reshape(-1, 1))

In this subsection you will split your data into: 
- Training data set
- Validation data set
- Testing data set

In [ ]:
# Split the data into data sets and display the number of records for each data set.
from sklearn.model_selection import train_test_split 
RANDOM_SEED = 92

# First split is 60/40
train_data, val_data = train_test_split(data_standardized, test_size=0.4, random_state=RANDOM_SEED)
#train_data, test_data = train_test_split(data_standardized, test_size=0.3, random_state=RANDOM_SEED)
train_target = train_data['Class']
train_data = train_data.drop(['Class'], axis=1)
# 2nd split is 50/50
val_data, test_data = train_test_split(val_data, test_size=0.5, random_state=RANDOM_SEED)
val_target = val_data['Class']
val_data =  val_data.drop(['Class'], axis=1)
test_target = test_data['Class']
test_data = test_data.drop(['Class'], axis=1)

print("Number of training records: " + str(len(train_target)))
print("Number of validation records: " + str(len(val_target)))
print("Number of testing records : " + str(len(test_target)))


### 3.2 Create the Keras machine learning model<a id="model"></a>

In this section you will create a Keras machine learning model and then train it.

First, import the related Python  packages that are needed in the subsequent steps.

In [ ]:
# Import Keras and related packages
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

Build and compile  the model. It's a single Dense layer with no hidden layers, softmax activation and Adam optimization

In [ ]:
model = Sequential()
model.add(Dense(2,  # output dim is 2, one per each class
                activation='softmax',
                input_dim=train_data.shape[1]))  # input dimension = number of features your data has

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### 3.3 Train the model<a id="train"></a>

Now, you can use the model with the training and validation data to mimimize the loss during training. Since the data is so unbalanced we give give more weight to the few fradulent transactions in the data set (we assign weights  that are the inverse of the relative frequency of fraudulent and normal transactions). This should take a  couple minutes.

In [ ]:
from keras.utils import to_categorical
class_weights = {0: 1., 1: len(train_target[train_target == 0])/len(train_target[train_target == 1])}
history = model.fit(train_data.values, to_categorical(train_target.values), class_weight=class_weights, epochs=20, batch_size=256, validation_data=(val_data.values, to_categorical(val_target.values))).history

Let's take a look at the validation loss by epoch to make sure that the model is converging

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')

Use **test data** to generate an evaluation report to check your **model quality**.

In [ ]:
from sklearn import metrics
# Evaluate your model.
predicted = model.predict_classes(test_data.values)

print("Evaluation report: \n\n%s" % metrics.classification_report(test_target.values, predicted))

**Note:** Your model is  predicting about 95% of the fraudulent transactions correctly and 98% of the normal ones. The **recall** of the fraudulent and normal transactions is probably the most important measure to keep an eye on (vs overall accuracy)

### 3.4 Examine the results <a id="evaluate"></a>

Let's look at a rudimentary cost model so we can see whether or not the model has saved us money. But first  we need to see the avg cost of both fraudulent and normal transactions

In [ ]:
mean_normal_transaction_amount = data[data.Class == 0]["Amount"].mean()
mean_fraudulent_transaction_amount = data[data.Class == 1]["Amount"].mean()
print('Average normal transaction amount %f' % mean_normal_transaction_amount)
print('Average fraudulent transaction amount %f' % mean_fraudulent_transaction_amount)

If we assume that we earn a trasnsaction fee of 2% for each valid transaction then a rudimentary cost model will works as follows:
- Every fraudulent transaction detected will save us approx $122.00

- Every fraudulent transaction missed  will cost us approx $122.00

- Every normal transaction classified as normal will earn  2% of $88

- Every normal transaction classified as fraudulent  will cost us 2% of $88  

Lets look at a confusion matrix to more insight into the results of the model. The top right quadrant has the  fraudulent transactions recognized by the model as normal and the bottom left has the normal transactions recognized as fraudulent. The bottom right and top left numbers represent the correctly  identified fraudulent and normal transactions respectively.

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
LABELS = ["Normal", "Fraudulent"]
conf_matrix = confusion_matrix(predicted, test_target)
plt.figure(figsize=(8, 8))
heatmap= sns.heatmap(conf_matrix, cmap="Blues",xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right')
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right')
plt.title("Confusion matrix")
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.show()

Based on this you would earn about ```$1.53```  per transaction without the model and ```$1.83``` per transaction with it  so  the model results in  a net positive effect on the bottom line. 

### 6. Summary and next steps <a id="summary"></a>    

You successfully completed this notebook! 
 
You learned how to use Keras to build a Logistic Regression model to predict credit card fraud. 

Check out our [Online Documentation](https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html) for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**David Carew** is a Developer Advocate at IBM.

Copyright © 2017, 2018 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>